In [162]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2
from math import floor
from joblib import Parallel, delayed
from sklearn.svm import SVC
import random
import os
from sklearn.metrics import precision_recall_fscore_support as full_score
import operator
from functools import reduce

In [ ]:
def g_mean(tp_0,tp_1):
    return (tp_0*tp_1)**(1/2)

In [ ]:
X_test = pd.read_csv("/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/DataGeneCausality/Tt/Erk.csv",sep=";").query('(Pvalue>=0.5) | (Pvalue<=0.01)')
del X_test['CauseGene']
del X_test['EffectGene']
del X_test['Replicate']
del X_test['Treatment']
X_test['Pvalue']=list(map(lambda x: 1 if x<=0.01 else 0, X_test['Pvalue']))
for column in X_test:
    if (column != "Pvalue"):
        X_test[column] = list(map(lambda x: x+1 , X_test[column]))
x_test_heads = []
for i in range(0,len(X_test.keys())):
    max_v=1
    for row in X_test[X_test.keys()[i]]:
        if (row>max_v-1):
            max_v+=row-(max_v-1)
    save_i=i+1
    x_test_heads.append("Node"+str(save_i)+"@"+str(max_v))
X_test.columns = x_test_heads
Y_test = X_test[X_test.keys()[0]]
TargetNode =X_test.keys()[0]
x_heads=x_test_heads

In [ ]:
X_test.head()

In [ ]:
import os

X_train=np.ndarray((0,len(X_test.keys())-1))
Y_train=np.ndarray((0,1))
for filename in os.listdir(main_directory):
#     if (filename!='batch_1.csv'):
#         continue
    X = pd.read_csv(main_directory+'/'+filename)
    # each cell can be -1, 0 or 1
    #Pre-processing data
    #X['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1), X['Pvalue']))
    #print(main_directory+filename)
    X['Pvalue']=list(map(lambda x: 1 if x<=0.05 else 0, X['Pvalue']))
    del X['Unnamed: 0']
    del X['TrueIndex']
    del X['CauseGene']
    del X['EffectGene']
    del X['Replicate']
    del X['Treatment']
    for column in X:
        if (column != "Pvalue"):
            X[column] = list(map(lambda x: x+1 , X[column]))
    X.columns = x_heads        
    Y_train=np.append(X.as_matrix([TargetNode]),Y_train,axis=0)
    #print(set(X['Node1@2']))
    del X[TargetNode]
    print(len(X.keys()))
    X_train=np.append(X.as_matrix(X.keys()),X_train,axis=0)
    x_train_heads = X.keys()

# making the new dataframe
X_train_df = pd.DataFrame(X_train,columns=x_train_heads)

In [ ]:
def ExistDseparator(TargetNode,Xi, Z, X, alpha):
    flagExist = False
    dsepSet=[]
    #counter=0
    #print_names(Z)
    for i in range(0,(2**len(Z))-1):
        IDsubsetZ_dec = i
        IDsubsetZ_bin = bin(IDsubsetZ_dec)
        subsetZ = getZsubset(IDsubsetZ_bin,Z)
        # no cache
        #print("from exist dseparator")
        dep = Dep(TargetNode,Xi,subsetZ, X, alpha)
        #print(subsetZ)
        #print(dep)
        if (dep==0):
            flagExist = True
            dsepSet = subsetZ
            break
    #print("Module exist d-separator: ",counter)
    return [flagExist,dsepSet]

In [163]:
def GetCondValues(z, condVars):
    if len(condVars)>1:
        maxz = reduce(operator.mul,[int(x['name'].split('@')[1]) for x in condVars],1)
        d = np.zeros(len(condVars))
        
        div_dim = maxz/int(condVars[0]['name'].split('@')[1])
        num2div = z
        for i in range(0,len(condVars)-1):
            d[i] = floor(num2div/div_dim)
            num2div=num2div%div_dim
            div_dim = div_dim/int(condVars[i+1]['name'].split('@')[1])
        d[i+1] = num2div
    else:
        d = [z]
    return d

In [ ]:
def Dep(TargetNode, Xi, CondVars, X, alpha):
    #print('in Dep')
    global data_c_size
    
    vvTarget = []
    for i in range(int(TargetNode['name'].split('@')[1])):
        vvTarget.append(i)
    
    vvXi = []
    for i in range(int(Xi['name'].split('@')[1])):
        vvXi.append(i)
    
    szCondVars = 1
    #print(CondVars)
    for column in CondVars:
        #print(column)
        szCondVars *= int(column['name'].split('@')[1])
    
    if data_c_size <= (5 * len(vvTarget)*len(vvXi)*(szCondVars)):
        return 1
    
    if (len(CondVars)==0):
        S = np.zeros((len(vvTarget),len(vvXi)))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                var1=np.array(TargetNode['data']) # Target Node
                var2=np.array(Xi['data'])
                # looking up for 0, 1 and 2
                #op1 = tf_mask_var(var1,vvTarget[i])
                op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                #op2 = tf_mask_var(var2,vvXi[j])
                op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                #S[i][j] = tf_dot_mult(op1,op2)
                S[i][j]=np.sum(op1*op2)
                #print(S[i][j])
        G2=0
        N = np.sum(S)
#         print(S)
#         print(N)
        Si =np.sum(S,axis=1)
        Sj =np.sum(S,axis=0)
        Df = ((len(vvTarget)-1)*(len(vvXi)-1))
#         print(S[np.where(S>0)])
        Dedf = len((S[np.where(S>0)]))\
                      - len(Si[np.where(Si>0)])\
                      - len(Sj[np.where(Sj>0)]) +1
#         print("Dedf before if: ",Dedf)
        if (Dedf<1):
            Dedf=1
            #print('Here is the trouble')
            #return 0
        #print(S)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                #print(i)
                #print(j)
                if (S[i][j]>0):
                    #print(Si[i])
                    #print(Sj[j])
                    G2 = G2 + S[i][j]*np.log((S[i][j])*N/(Si[i]*Sj[j]))
                    #print(G2)
        G2 = 2*G2
        
    else: # test conditional dependency
        
            
        S = np.zeros((len(vvTarget),len(vvXi),szCondVars))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    condValue = GetCondValues(k,CondVars)
                    #print(condValue)
                    flagDataCondVars = np.ones(len(Xi['data']))
                    for l in range(0,len(CondVars)):
                        X_l = CondVars[l]
                        flagDataCondVars = flagDataCondVars*np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l['data'])))
                        #op1 = tf_mask_var(X_l['data'],condValue[l])
                        #flagDataCondVars = np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l.tolist())))
                        #flagDataCondVars = flagDataCondVars*op1
                    var1=np.array(TargetNode['data'])
                    var2=np.array(Xi['data'])
                    # looking up for -1, 0 and 1
                    op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                    #op1 = tf_mask_var(var1,vvTarget[i])
                    #print(op1)
                    #op2 = tf_mask_var(var2,vvXi[j])
                    op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                    #print(op2)
                    #print(op1*op2*flagDataCondVars)
                    S[i][j][k]=np.sum(op1*op2*flagDataCondVars)
        G2 = 0
        
        Sjk = np.sum(S,axis=0)
        Sik = np.sum(S,axis=1)
        Sk = np.sum(Sjk,axis=0)
        
        #Sjk = sum(S,1);
        #Sik = sum(S,2);
        #Sk = sum(Sjk,2); 
        Dedf = len(S[np.where(S>0)]) -\
                len(Sik[np.where(Sik>0)]) -\
                len(Sjk[np.where(Sjk>0)]) +\
                len(Sk[np.where(Sk>0)])
        if Dedf<1:
            Dedf=1
            #print('No, here is the trouble')
            #return 0
        
        # compute G2 statistic
        #print(S.shape)
        #print(Sjk.shape)
        #print(Sik.shape)
        #print(Sk.shape)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    if (S[i][j][k]>0):
                        #print(i,j,k)
                        #test_op =  Sk[k]
                        #test_op = Sik[i][k]
                        #test_op = Sjk[j][k]
                        G2=G2+(S[i][j][k] * np.log(\
                                    (S[i][j][k] * Sk[k])/\
                                    (Sik[i][k] * Sjk[j][k])))
        G2 = 2*G2
    
    assoc = (alpha - (1 - chi2.cdf(G2,Dedf)))/alpha
    #print(G2)
    #print(Dedf)
    #print(assoc)
    if assoc<0:
        assoc=0
#     print("Target Node: ",TargetNode['name'])
#     print("Node XI: ", Xi['name'])
#     print("\tG2:",G2)
#     print("\tDedf: ",Dedf)
#     print("\tAssoc: ",assoc)
#     print()
#     print()
    return assoc

In [ ]:
def getZsubset(bin_id,Z):
    bin_str=str(bin_id[::-1])
    Zsubset=[]
    for i in range(0,len(bin_str)):
        if bin_str[i]=='1':
            Zsubset.append(Z[i])
    return Zsubset

In [ ]:
def arrayUniverse(TargetNode,X):
    Universe = []
    for key in X:
        #print("Key in X:",key)
        #print("Target Node is: ",TargetNode)
        if (key!=TargetNode):
            append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
            Universe.append(append_dict)
    return Universe

In [ ]:
def arrayX(X):
    returnable=[]
    for key in X:
        #tagged_c = MB_Column(X[key].copy(deep=True).tolist(),key)
        append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
        returnable.append(append_dict)
    return returnable

In [ ]:
def print_names(dict_list):
    print()
    for item in dict_list:
        print(item['name'],end=" ")
    print()

In [ ]:
def MinAssoc(TargetNode, Xi,Z, fixedCondVars, X, alpha):
    
    min_assoc=999
    #counter=0
    #print(fixedCondVars)
    if len(Z)==0:
        #print("from minassoc1")
        #print(fixedCondVars)
        min_assoc = Dep(TargetNode, Xi, fixedCondVars, X, alpha)
        #counter+=1
        subsetZ_min_assoc = fixedCondVars
        #print(min_assoc)
    else:
        #print(2**len(Z)-1)
        for IDsubsetZ_dec in range(0,2**len(Z)-1):
            IDsubsetZ_bin = bin(IDsubsetZ_dec)
            subsetZ = getZsubset(IDsubsetZ_bin,Z)
            #print(len(Xi))
            #print("from minassoc2")
            #print(IDsubsetZ_dec)
            #print(subsetZ)
            subsetZ_assoc=Dep(TargetNode, Xi, fixedCondVars+subsetZ,X,alpha)
            #counter+=1
            #print(subsetZ_assoc[IDsubsetZ_dec])
            if subsetZ_assoc < min_assoc:
                min_assoc = subsetZ_assoc
                subsetZ_min_assoc = fixedCondVars + subsetZ
                if (min_assoc==0):
                    break
    #print("Min Assoc module: ",counter)
    return min_assoc

In [ ]:
def MaxMinHeuristic(TargetNode, CPC, Universe, X, alpha):
    F=[]
    assocF=-1
    Z = CPC
    fixedCondVars = []
    if (len(CPC)>0):
        Z = CPC[0:-1]           # all but the last one   
        fixedCondVars = [CPC[-1]] # we use last one
    
    for i in range(len(Universe)-1,-1,-1):
    #for i in range(0,len(Universe)):
        if (len(Universe[i])==0):
            continue
        minAssoc = MinAssoc(TargetNode,Universe[i], Z,fixedCondVars, X,alpha)
        #print("\nFor "+Universe[i]['name']+"the minAssoc value was: "+str(minAssoc))
        if minAssoc <= 0.5:
            #Universe[i]=[]
            Universe.pop(i)
            continue
        if minAssoc>assocF:
            assocF = minAssoc
            F=Universe[i]
        
        #print(F)
    return [F,assocF,Universe]

In [ ]:
def print_CPC(CPC):
    for i in CPC:
        print(i['name']+" ", end="")

In [ ]:
def MMPC_(TargetNode,Universe,X,alpha):
    
    CPC=[]
    #print(TargetNode)
    # Phase I: Foward
    print("Entering Phase I")
    print("MMPC_beggining: \n"+str(len(Universe)))
    while len(Universe)>0:
        CPC_old = list(CPC) # copy
        maxminheur=MaxMinHeuristic(TargetNode,CPC,list(Universe),X,alpha)
        # maxminheur = [F, assocF, Universe]
        F = maxminheur[0]
        assocF = maxminheur[1]
        #print(assocF)
        Universe = maxminheur[2]
        #print("Universe printing line")
        #print(Universe)
        #print("\n=============================")
        #print(CPC)
        if assocF > 0:
            CPC.append(F)
            indF=Universe.index(F)
            Universe.pop(indF)
        #if (len(CPC)==len(CPC_old)) or (len(CPC)>0.3*(len(Universe)-1)):
        if (len(CPC)==len(CPC_old)):
            break
#         print("Universe actual size:")
#         print(len(Universe))
#         print("CPC actual size:")
#         print(len(CPC))
#         print("CPC contents:")
#         print_CPC(CPC)
    
    # Phase 2: Backward
    print("\nExiting MMPC_")
#     CPC=CPC[::-1]
#     if len(CPC)>1:
#         Z=list(CPC)
#         for i in range(len(CPC)-1,-1,-1):
#             # index is i
#             Z.pop(i)
#             #print("Analyzing D-separator for ",CPC[i]['name'])
#             if ExistDseparator(TargetNode,CPC[i],Z,X,alpha)[0] == True:
#                 #print("it did exist! removing from cpc.")
#                 CPC.pop(i)
#     return symmetry_test_v2(TargetNode,CPC,X,alpha)
    return CPC

In [ ]:
def MMPC(TargetNode, X, alpha):
    # The universe will be an array of DataFrame Columns
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    for column in X:
        if (column['name']==TargetNode):
            TargetNode = column
            break
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [ ]:
def MMPC_Parallel(arg):
    TargetNode, X, alpha = arg
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    for column in X:
        if (column['name']==TargetNode):
            TargetNode = column
            break
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [ ]:
def getKeysPartition(targetNode, data_keys,n_partitions):
    n_partitions = int(np.log2(len(data_keys)))*2*2  # log2 ( number of variables ) divided by 2
    partitions = []
    partition_diff = int(int(len(data_keys)/n_partitions)/2)
    for i in range(0,n_partitions):
        if (i==0):
            partitions.append(list(data_keys[-partition_diff::])+list(data_keys[0:int(len(data_keys)/n_partitions)]))
            #partitions.append(list(data_keys[0:int(len(data_keys)/n_partitions)]))
            continue
        if (i<n_partitions-1):
            partitions.append(list(data_keys[-partition_diff+int(len(data_keys)/n_partitions)*i:int(len(data_keys)/n_partitions)*(i+1)]))
            #partitions.append(list(data_keys[int(len(data_keys)/n_partitions)*i:int(len(data_keys)/n_partitions)*(i+1)]))
            continue
        if (i==n_partitions-1):
            partitions.append(list(data_keys[-partition_diff+int(len(data_keys)/n_partitions)*i::]))
            #partitions.append(list(data_keys[int(len(data_keys)/n_partitions)*i::]))
            
    for i in range(0,n_partitions):
        for j in range(len(partitions[i])-1,-1,-1):
            if (partitions[i][j]==targetNode):
                partitions[i].pop(j)
        partitions[i] = partitions[i] + [targetNode]
    return partitions,n_partitions

In [ ]:
%%time
alpha=0.05
print("======================================================================================")
print("Init Genetic Algorithm")
print("======================================================================================")
main_directory= '/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Balanced_Batches/censored_Erk/'
if not os.path.exists(main_directory):
    print("Bad routing.")
PCs=[]
for filename in os.listdir(main_directory)[:1]:
    print("Analizing "+filename)
    X = pd.read_csv(main_directory+'/'+filename)
    #print(X.keys())
    # each cell can be -1, 0 or 1
    #Pre-processing data
    #X['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1), X['Pvalue']))
    X['Pvalue']=list(map(lambda x: 1 if x<=0.01 else 0, X['Pvalue']))
    #del X['Unnamed: 0']
    #del X['TrueIndex']
    del X['CauseGene']
    del X['EffectGene']
    del X['Replicate']
    del X['Treatment']
    PC=[]
    for column in X:
        if (column != "Pvalue"):
            X[column] = list(map(lambda x: x+1 , X[column]))
    x_heads = []
    for i in range(0,len(X.keys())):
        max_v=1
        for row in X[X.keys()[i]]:
            if (row>max_v-1):
                max_v+=row-(max_v-1)
        save_i=i+1
        x_heads.append("Node"+str(save_i)+"@"+str(max_v))
    X.columns = x_heads
    data_c_size = len(X[X.keys()[1]])
    data_vars = [0,1,2,3]
    print("======================================================================================")
    TargetNode =X.keys()[0]
    n_partitions = 0
    keys_partitions, n_partitions = getKeysPartition(TargetNode,X.keys(),n_partitions)
    arg_instances = []
    for i in range(0,n_partitions):
        arg_instances.append((TargetNode,X[keys_partitions[i]],alpha))
    PC=Parallel(n_jobs=n_partitions)(map(delayed(MMPC_Parallel),arg_instances))
    PC_dict={}
    PC_dict['pcs']=PC
    PC_dict['filename']=filename
    PCs.append(PC_dict)
# for after
n_parallel_forks = n_partitions
    

In [ ]:
initial_universe=[]
for PC_dict in PCs:
    PC_train={}
    PC_train['dataset']=PC_dict['filename']
    PC_train['supercpc']=[]
    for pc_subset in PC_dict['pcs']:
        if (len(pc_subset)==0):
            continue
        for e in pc_subset:
            PC_train['supercpc'].append(e['name'])
    PC_train['supercpc']=list(set(PC_train['supercpc']))
    initial_universe.append(PC_train)

In [ ]:
import time
PCs_mean_scores=[]
for i in range(0,1000,100):
    if (i==0):
        continue
    PC_mean_score=[]
    for PC_set in initial_universe:    
        start = time.time()
        print("For "+PC_set['dataset'])
        print("CPC of length "+str(len(PC_set['supercpc'])))
        PC_mean_score.append(CandidateScore(PC_set,i)['score'])    
        end = time.time()
        print("Time elapsed: "+str(end - start)+" seconds")
        print("=================================================================")
    print("Mean score for Intercept == "+str(i)+" was: "+str(sum(PC_mean_score)/len(PC_mean_score)))
    print("/////////////////////////////////////////////////////////////////////")
    PCs_mean_scores.append(sum(PC_mean_score)/len(PC_mean_score))
    print("/////////////////////////////////////////////////////////////////////")

In [ ]:
safeCopyPcs = list(PCs)

In [ ]:
%%time
poblation_gen_init = []
Y_pred = []
Y_pred = X_test[X_test.keys()[0]]

for PC_dict in PCs:
    for pc_subset in PC_dict['pcs']:
        # Structure for poblation in genetic algorithm will be like this
        # ->cpc
        # ->score
        PobDict={}
        PobDict['cpc']=[]
        for e in pc_subset:
        PobDict['filenames']=[]
        PobDict['filenames'].append(filename)
        #PobDict['score']=clf.score(X_pred,Y_pred)
        poblation_gen_init.append(PobDict)
        #print(PobDict['score'])

        
        

In [ ]:
%%time
# Parallel processing
scored_init_pob = Parallel(n_jobs=n_parallel_forks)(map(delayed(CandidateScore),poblation_gen_init))

In [ ]:
def CandidateScore(PobDict,intercept_scaling):
    global X_test
    global x_heads

    # training and scoring
    X_train_df = pd.read_csv(main_directory+PobDict['dataset'])
    del X_train_df['CauseGene']
    del X_train_df['EffectGene']
    del X_train_df['Replicate']
    del X_train_df['Treatment']
    X_train_df['Pvalue']=list(map(lambda x: 1 if x<=0.01 else 0, X_train_df['Pvalue']))
    x_heads[0]='Node1'
    X_train_df.columns=x_heads
    X_test.columns=x_heads
    #print(X_train_df.keys())
    Y_train=X_train_df["Node1"]
    
    
    # training classifier
    clf = LinearSVC(loss='hinge', C=0.001, intercept_scaling=intercept_scaling)
    clf.fit(X_train_df[PobDict['supercpc']],Y_train)
    
    # get predicted values for TargetNode
    #Y_pred=clf.predict(X_test[PobDict['supercpc'][:10]])
    
    # get score values
    precision = []
    Y_pred_0 = clf.predict(X_test.query('Node1 ==0')[PobDict['supercpc']])
    precision.append(Y_pred_0.tolist().count(0)/len(Y_pred_0))
    Y_pred_1 = clf.predict(X_test.query('Node1 ==1')[PobDict['supercpc']])
    precision.append(Y_pred_1.tolist().count(1)/len(Y_pred_1))
    
    #precision.append(clf.score(X_test.query('Node1 ==0')[PobDict['supercpc'][:10]],X_test.query('Node1 ==0')[['Node1']]))
    #precision.append(clf.score(X_test.query('Node1 ==1')[PobDict['supercpc'][:10]],X_test.query('Node1 ==1')[['Node1']]))
    print('precision for class 0: '+str(precision[0]))
    print('precision for class 1: '+str(precision[1]))
    
    print("G-mean score: "+str(g_mean(precision[0],precision[1])))
    #print(clf.score(X_test[PobDict['supercpc'][:10]],Y_test))
    PobDict['score']=g_mean(precision[0],precision[1])
    return PobDict

In [ ]:
safeCopyScoredPoblation = list(scored_init_pob)

In [ ]:
# sorting the poblation
from operator import itemgetter
poblation_sorted = sorted(scored_init_pob, key=itemgetter('score'), reverse=True)

In [ ]:
# logging results
# unfinished
counter_p = 1
for cpc in poblation_sorted:    
    print("===========================================")
    print("CPC n"+str(counter_p)+" with score: "+str(cpc['score']))
    counter_p=counter_p+1
    print(cpc['cpc'])
    print("===========================================")
    
    print()

In [ ]:
def Node_inside(cpc, cpc_node):
    for e in cpc:
        if (e==cpc_node):
            return True
    return False

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
cross_parent_pob = list(train_test_split(poblation_sorted,train_size=0.05))

In [ ]:
len(cross_parent_pob[0])

In [ ]:
a=list(range(0,10,2))
a


In [ ]:
import itertools

test_list_1=[[1,2,3,4],[5,6,7,8]]


test_list_1 = list(itertools.chain.from_iterable(test_list_1))
test_list_1


In [ ]:
len(poblation)

In [ ]:
%%time
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
# crossover
# we will take the alpha % of the best poblation to do the crossover
alpha_crossover = 0.50

# we will make beta childs per couple
beta_crossover = 1

# probability of taking an atribute present in both groups
thetha_crossover = 0.95

# probability of taking an atribute not present in both groups
sigma_crossover = 0.40

# poblation selected
crossover_parent_poblation = list(train_test_split(poblation_sorted,train_size=alpha_crossover))[0]

# crossover
crossover_child_poblation=[]

crossover_parallel_results=[]

step_parents = 4

for cpc_a_counter in range(0,len(crossover_parent_poblation),step_parents):
    if (cpc_a_counter+step_parents>=len(crossover_parent_poblation)):
        break
    
    
    cpc_a = {}
    cpc_a['cpc']=[]
    cpc_a['filenames']=[]
    # appending the fathers from a
    
        
    for cpc_group_a_counter in range(cpc_a_counter,cpc_a_counter+step_parents):
        for cpc_name in crossover_parent_poblation[cpc_group_a_counter]['cpc']:
            cpc_a['cpc'].append(cpc_name)
        for cpc_filename in crossover_parent_poblation[cpc_group_a_counter]['filenames']:
            cpc_a['filenames'].append(cpc_filename)
    
    for cpc_b_counter in range(0,len(crossover_parent_poblation)):
        if (cpc_b_counter+step_parents>=len(crossover_parent_poblation)):
            break
        
        
        cpc_b={}
        cpc_b['cpc']=[]
        cpc_b['filenames']=[]
        #appending fathers from b
        for cpc_group_b_counter in range(cpc_b_counter,cpc_b_counter+step_parents):
            for cpc_name in crossover_parent_poblation[cpc_group_b_counter]['cpc']:
                cpc_b['cpc'].append(cpc_name)
            for cpc_filename in crossover_parent_poblation[cpc_group_b_counter]['filenames']:
                cpc_b['filenames'].append(cpc_filename)
    
        if (cpc_a_counter==cpc_b_counter):
            continue
        for crossover_counter in range(0,beta_crossover):
            # first, we select the atributes present in both cpcs
            PobDict={}
            
            cross_child=[]
            for node_a in cpc_a['cpc']:
                for node_b in cpc_b['cpc']:
                    if (node_a==node_b):
                        # diversity conditioning
                        prob_res = random.random()
                        if (prob_res<thetha_crossover):
                            cross_child.append(node_a)
                            continue
            # for parent a
            for node_a in cpc_a['cpc']:
                if (Node_inside(cross_child,node_a)==False):
                    prob_res = random.random()
                    if (prob_res<sigma_crossover):
                        cross_child.append(node_a)
            # for parent b
            for node_b in cpc_b['cpc']:
                if (Node_inside(cross_child,node_b)==False):
                    prob_res = random.random()
                    if (prob_res<sigma_crossover):
                        cross_child.append(node_b)
            
            if (len(cross_child)==0):
                continue
            PobDict['cpc']=sorted(cross_child)
            
            
            # filenames
            PobDict['filenames']=[]
            for filename in cpc_a['filenames']:
                PobDict['filenames'].append(filename)
            for filename in cpc_b['filenames']:
                try:
                    try_var=PobDict['filenames'].index(filename)
                except:
                    PobDict['filenames'].append(filename)
            crossover_child_poblation.append(PobDict)
scored_child_pob = Parallel(n_jobs=n_parallel_forks)(map(delayed(CandidateScore),crossover_child_poblation))

In [ ]:
child_poblation_sorted = sorted(scored_child_pob, key=itemgetter('score'), reverse=True)

In [ ]:
180*0.4

In [ ]:
len(child_poblation_sorted[:int(poblation_size*(1-new_pob_perc))-1])

In [ ]:
from sklearn.utils import shuffle

new_pob_perc=0.40
# ideally, we will use 180 as cap for the poblation
poblation_size=180
for genetic_algorithm_counter in range(0,100):
    new_pob = []
    for e in child_poblation_sorted[:int(poblation_size*(1-new_pob_perc))-1]:
        new_pob.append(e)
    for e in list(shuffle(poblation_sorted))[:int(poblation_size*new_pob_perc)-1]:
        new_pob.append(e)
    # poblation selected
    crossover_parent_poblation = list(train_test_split(new_pob,train_size=alpha_crossover))[0]

    # crossover
    crossover_child_poblation=[]

    crossover_parallel_results=[]

    for cpc_a_counter in range(0,len(crossover_parent_poblation)):
        cpc_a = crossover_parent_poblation[cpc_a_counter]
        for cpc_b_counter in range(0,len(crossover_parent_poblation)):
            cpc_b = crossover_parent_poblation[cpc_b_counter]
            if (cpc_a_counter==cpc_b_counter):
                continue
            for crossover_counter in range(0,beta_crossover):
                # first, we select the atributes present in both cpcs
                PobDict={}

                cross_child=[]
                for node_a in cpc_a['cpc']:
                    for node_b in cpc_b['cpc']:
                        if (node_a==node_b):
                            # diversity conditioning
                            prob_res = random.random()
                            if (prob_res<thetha_crossover):
                                cross_child.append(node_a)
                                continue
                # for parent a
                for node_a in cpc_a['cpc']:
                    if (Node_inside(cross_child,node_a)==False):
                        prob_res = random.random()
                        if (prob_res<sigma_crossover):
                            cross_child.append(node_a)
                # for parent b
                for node_b in cpc_b['cpc']:
                    if (Node_inside(cross_child,node_b)==False):
                        prob_res = random.random()
                        if (prob_res<sigma_crossover):
                            cross_child.append(node_b)

                if (len(cross_child)==0):
                    continue
                PobDict['cpc']=sorted(cross_child)


                # filenames
                PobDict['filenames']=[]
                for filename in cpc_a['filenames']:
                    PobDict['filenames'].append(filename)
                for filename in cpc_b['filenames']:
                    try:
                        try_var=PobDict['filenames'].index(filename)
                    except:
                        PobDict['filenames'].append(filename)
                crossover_child_poblation.append(PobDict)
    scored_child_pob = Parallel(n_jobs=n_parallel_forks*2)(map(delayed(CandidateScore),crossover_child_poblation))
    child_poblation_sorted = sorted(scored_child_pob+new_pob, key=itemgetter('score'), reverse=True)
    print("Round n "+str(genetic_algorithm_counter))
    print(child_poblation_sorted[0]['filenames'])
    print(child_poblation_sorted[0]['cpc'])
    print(child_poblation_sorted[0]['score'])

In [ ]:
for e in child_poblation_sorted:
    print("==============================================================================")
    print(e['filenames'])
    print(e['cpc'])
    print(e['score'])
    print("==============================================================================")
    print()

In [ ]:
len(poblation_sorted)

In [ ]:
%%time
alpha=0.05
# Initial vectors setup
#const_vectors = setup_const_vectors(data_vars,data_c_size)
#for i in range(0,len(X.keys())):    
#for i in range(0,1):  

print("======================================================================================")
TargetNode =X.keys()[0]
n_partitions = 0
keys_partitions, n_partitions = getKeysPartition(TargetNode,X.keys(),n_partitions)
arg_instances = []
for i in range(0,n_partitions):
    arg_instances.append((TargetNode,X[keys_partitions[i]],alpha))
# we know that core number is 8
n_cores=-1
if (n_partitions>8):
    n_cores = 8
else:
    n_cores=n_partitions
PC=(map(MMPC_Parallel,arg_instances))

total_PC = []
for pc_element in PC:
    total_PC=total_PC+pc_element
TargetNodeDict={}
TargetNodeDict['name'] = TargetNode
TargetNodeDict['data'] = X[TargetNode]
print("Target Node was: "+TargetNodeDict['name'])
for i in range(len(total_PC)-1,-1,-1):
    for j in range(len(total_PC)-1,-1,-1):
        if i != j:
            if total_PC[i]['name']==total_PC[j]['name']:
                total_PC.pop(i)
                break
print("PC set is :")
for j in total_PC:
    print(j['name'])
    
clf = LinearSVC(random_state=0)
X_val=[]
Y=[]
X_arr=arrayX(X)
for i in range(0,data_c_size):
    d_row = []
    for pc_column in X_arr:
        d_row.append(pc_column['data'][i])
    X_val.append(d_row)
for i in range(0,data_c_size):
    Y.append(TargetNodeDict['data'][i])
clf.fit(X_val[:-1000],Y[:-1000])
print(clf.score(X_val[-1000:],Y[-1000:]))

In [ ]:
#Phase 2: Backward
def symmetry_test(TargetNode,CPC,X,alpha):
    print("\nEntering Phase II")
    if len(CPC)>1:
        Z=list(CPC)
        for i in range(len(CPC)-1,-1,-1):
            # index is i
            Z.pop(i)
            print("Analyzing D-separator for ",CPC[i]['name'])
            if ExistDseparator(TargetNode,CPC[i],Z,X,alpha)[0] == True:
                print("it did exist! removing from cpc.")
                CPC.pop(i)
    return CPC

In [ ]:
# modified symmetry  test
def symmetry_test_v2(TargetNode, CPC, X,alpha):
    print("\nEntering Phase II")
    if (len(CPC)>1):
        Z=list(CPC)
        while len(Z)>0:
            # Will test all vs the Target, and select the lowest
            minDep=999
            minDepIndex=-1
            for i in range(0,len(Z)):
                zCopy = list(Z)
                zCopy.pop(i)
                auxDep = Dep(TargetNode,Z[i],zCopy,X,alpha)
                if (minDep>auxDep):
                    minDep = auxDep
                    minDepIndex=i
            # Find index in CPC
            CPCindex=-1
            for i in range(0,len(CPC)):
                if (CPC[i]['name']==Z[minDepIndex]['name']):
                    CPCindex=i
                    break
            
            # Remove from Z
            Z.pop(minDepIndex)
            print_names(Z)
            print("Analyzing D-separator for ",CPC[CPCindex]['name'])
            if ExistDseparator(TargetNode,CPC[CPCindex],Z,X,alpha)[0] == True:
                print("it did exist! removing from cpc.")
                CPC.pop(CPCindex)
    return CPC

In [ ]:
def backward_dep_parallel(TargetNode, CPC, X,alpha):    
    print("\nEntering Phase II")
    if (len(CPC)>1):
        Z=list(CPC)
        while len(Z)>0:
            # Will test all vs the Target, and select the lowest
            minDep=999
            minDepIndex=-1
            arg_instances = []
            for i in range(0,len(Z)):
                zCopy = list(Z)
                zCopy.pop(i)
                arg_instances.append((TargetNode,Z[i],zCopy,X,alpha))
                #auxDep = Dep(TargetNode,Z[i],zCopy,X,alpha)
            depResults = Parallel(n_jobs=8)(map(delayed(Dep_Parallel),arg_instances))
            for i in range(0,len(depResults)):
                if (minDep>depResults[i]):
                    minDep = depResults[i]
                    minDepIndex=i
            # Find index in CPC
            CPCindex=-1
            for i in range(0,len(CPC)):
                if (CPC[i]['name']==Z[minDepIndex]['name']):
                    CPCindex=i
                    break
            
            # Remove from Z
            Z.pop(minDepIndex)
            print_names(Z)
            print("Analyzing D-separator for ",CPC[CPCindex]['name'])
            if ExistDseparator(TargetNode,CPC[CPCindex],Z,X,alpha)[0] == True:
                print("it did exist! removing from cpc.")
                CPC.pop(CPCindex)
    return CPC

In [ ]:
def Dep_Parallel(args):
    #print('in Dep')
    TargetNode, Xi, CondVars, X, alpha = args
    global data_c_size
    
    vvTarget = []
    for i in range(int(TargetNode['name'].split('@')[1])):
        vvTarget.append(i)
    
    vvXi = []
    for i in range(int(Xi['name'].split('@')[1])):
        vvXi.append(i)
    
    szCondVars = 1
    #print(CondVars)
    for column in CondVars:
        #print(column)
        szCondVars *= int(column['name'].split('@')[1])
    
    if data_c_size <= (5 * len(vvTarget)*len(vvXi)*(szCondVars)):
        return 1
    
    if (len(CondVars)==0):
        S = np.zeros((len(vvTarget),len(vvXi)))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                var1=np.array(TargetNode['data']) # Target Node
                var2=np.array(Xi['data'])
                # looking up for 0, 1 and 2
                #op1 = tf_mask_var(var1,vvTarget[i])
                op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                #op2 = tf_mask_var(var2,vvXi[j])
                op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                #S[i][j] = tf_dot_mult(op1,op2)
                S[i][j]=np.sum(op1*op2)
                #print(S[i][j])
        G2=0
        N = np.sum(S)
#         print(S)
#         print(N)
        Si =np.sum(S,axis=1)
        Sj =np.sum(S,axis=0)
        Df = ((len(vvTarget)-1)*(len(vvXi)-1))
#         print(S[np.where(S>0)])
        Dedf = len((S[np.where(S>0)]))\
                      - len(Si[np.where(Si>0)])\
                      - len(Sj[np.where(Sj>0)]) +1
#         print("Dedf before if: ",Dedf)
        if (Dedf<1):
            Dedf=1
            #print('Here is the trouble')
            #return 0
        #print(S)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                #print(i)
                #print(j)
                if (S[i][j]>0):
                    #print(Si[i])
                    #print(Sj[j])
                    G2 = G2 + S[i][j]*np.log((S[i][j])*N/(Si[i]*Sj[j]))
                    #print(G2)
        G2 = 2*G2
        
    else: # test conditional dependency
        
            
        S = np.zeros((len(vvTarget),len(vvXi),szCondVars))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    condValue = GetCondValues(k,CondVars)
                    #print(condValue)
                    flagDataCondVars = np.ones(len(Xi['data']))
                    for l in range(0,len(CondVars)):
                        X_l = CondVars[l]
                        flagDataCondVars = flagDataCondVars*np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l['data'])))
                        #op1 = tf_mask_var(X_l['data'],condValue[l])
                        #flagDataCondVars = np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l.tolist())))
                        #flagDataCondVars = flagDataCondVars*op1
                    var1=np.array(TargetNode['data'])
                    var2=np.array(Xi['data'])
                    # looking up for -1, 0 and 1
                    op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                    #op1 = tf_mask_var(var1,vvTarget[i])
                    #print(op1)
                    #op2 = tf_mask_var(var2,vvXi[j])
                    op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                    #print(op2)
                    #print(op1*op2*flagDataCondVars)
                    S[i][j][k]=np.sum(op1*op2*flagDataCondVars)
        G2 = 0
        
        Sjk = np.sum(S,axis=0)
        Sik = np.sum(S,axis=1)
        Sk = np.sum(Sjk,axis=0)
        
        #Sjk = sum(S,1);
        #Sik = sum(S,2);
        #Sk = sum(Sjk,2); 
        Dedf = len(S[np.where(S>0)]) -\
                len(Sik[np.where(Sik>0)]) -\
                len(Sjk[np.where(Sjk>0)]) +\
                len(Sk[np.where(Sk>0)])
        if Dedf<1:
            Dedf=1
            #print('No, here is the trouble')
            #return 0
        
        # compute G2 statistic
        #print(S.shape)
        #print(Sjk.shape)
        #print(Sik.shape)
        #print(Sk.shape)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    if (S[i][j][k]>0):
                        #print(i,j,k)
                        #test_op =  Sk[k]
                        #test_op = Sik[i][k]
                        #test_op = Sjk[j][k]
                        G2=G2+(S[i][j][k] * np.log(\
                                    (S[i][j][k] * Sk[k])/\
                                    (Sik[i][k] * Sjk[j][k])))
        G2 = 2*G2
    
    assoc = (alpha - (1 - chi2.cdf(G2,Dedf)))/alpha
    #print(G2)
    #print(Dedf)
    #print(assoc)
    if assoc<0:
        assoc=0
#     print("Target Node: ",TargetNode['name'])
#     print("Node XI: ", Xi['name'])
#     print("\tG2:",G2)
#     print("\tDedf: ",Dedf)
#     print("\tAssoc: ",assoc)
#     print()
#     print()
    return assoc

In [ ]:
clf = LinearSVC(random_state=0)
X_val=[]
Y=[]
X=arrayX(X)
for i in range(0,data_c_size):
    d_row = []
    for pc_column in X:
        d_row.append(pc_column['data'][i])
    X_val.append(d_row)
for i in range(0,data_c_size):
    Y.append(TargetNodeDict['data'][i])
clf.fit(X_val[:-1000],Y[:-1000])
print(clf.score(X_val[-1000:],Y[-1000:]))

In [ ]:
string_arr = ['hello','there']